MongoDB Queries for Employee Data Analysis. Ridwan Badamasi

This notebook contains Python code to connect to a MongoDB database and perform a series of queries on an employee dataset which was generated using the Faker library to simulate 10,000 employee records.

In [1]:
pip install pymongo faker

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



Setup and Connection
First, we'll import the necessary libraries, generate data with Faker and establish a connection to the MongoDB database.

In [2]:
from faker import Faker
import random
from pymongo import MongoClient
from datetime import datetime, timedelta

fake = Faker() 

employees_collection = None
try:
    client = MongoClient("mongodb://localhost:27017/") 
    db = client.Employee_data
    employees_collection = db.employees  
    print("Successfully connected to MongoDB.")

except Exception as e:
    print(f"Error connecting to MongoDB: {e}")


    # Lists for predefined data to ensure some consistency in the generated data
departments = ["Engineering", "Sales", "Marketing", "Finance", "Human Resources", "IT", "Operations"]
skills_list = ["Python", "JavaScript", "MongoDB", "SQL", "React", "Node.js", "Java", "C++", "AWS", "Azure", "Docker", "Kubernetes", "Data Analysis", "Project Management"]

def generate_employee_document(employee_id):
    """
    Generates a single fake employee document.
    """
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = f"{first_name.lower()}.{last_name.lower()}@{fake.domain_name()}"
    department = random.choice(departments)
    
    if department == "Engineering":
        positions = ["Software Engineer", "Senior Developer", "DevOps Engineer", "Data Scientist"]
        position = random.choice(positions)
    elif department == "Sales":
        positions = ["Sales Representative", "Account Manager", "Sales Manager"]
        position = random.choice(positions)
    # ... (other department logic for positions)
    else: 
        positions = ["Operations Analyst", "Project Manager", "Logistics Coordinator"]
        position = random.choice(positions)

    hire_date = fake.date_time_between(start_date="-15y", end_date="now")
    years_experience = datetime.now().year - hire_date.year
    last_promotion = fake.date_time_between(start_date=hire_date, end_date="now") if years_experience > 2 else hire_date

    skills = random.sample(skills_list, k=random.randint(2, 5))
    salary = random.randint(50000, 150000)
    performance_rating = round(random.uniform(2.0, 5.0), 1)

    return {
        "employee_id": f"EMP{employee_id:05d}",
        "first_name": first_name,
        "last_name": last_name,
        "email": email,
        "department": department,
        "position": position,
        "salary": salary,
        "years_experience": years_experience,
        "performance_rating": performance_rating,
        "skills": skills,
        "hire_date": hire_date,
        "last_promotion": last_promotion,
        "is_remote": fake.boolean(),
        "address": {
            "city": fake.city(),
            "state": fake.state_abbr(),
            "zip_code": fake.zipcode(),
        }
    }

def generate_and_insert_employees(num_employees):
    """
    Generates a specified number of employee documents and inserts them into MongoDB.
    """
    employees_to_insert = []
    for i in range(1, num_employees + 1):
        employee_doc = generate_employee_document(i)
        employees_to_insert.append(employee_doc)
    
    try:
        employees_collection.insert_many(employees_to_insert) #this function inserts multiple documents at once
        print(f"Successfully inserted {num_employees} employee documents.")
    except Exception as e:
        print(f"An error occurred: {e}")

# This function populates the db & will only run if connection is successful.
if employees_collection is not None:
    generate_and_insert_employees(10000)
else:
    print("Database connection failed, skipping data generation.")

Successfully connected to MongoDB.
Successfully inserted 10000 employee documents.


Q1. **High Performers Query:** Find all employees with performance rating >= 4.0 **AND** salary > 80,000. Return only their name, department, salary, and performance rating.

In [ ]:
if employees_collection is None:
    print("Database connection failed. Cannot perform query.")

else:
    print("Database connection successful. Proceeding with queries.")

query = {
    "performance_rating": {"$gte": 4.0}, #The $gte (greater than or equal to) operator is used to filter for a performance rating of at least 4.0.
    "salary": {"$gt": 80000}  #The $gt (greater than) operator is used to filter for a salary greater than $80,000.
}

#The projection document is used to specify which fields should be returned. 
# We set each desired field to 1 and explicitly exclude the default _id field by setting it to 0.
projection = {
    "first_name": 1,
    "last_name": 1,
    "department": 1,
    "salary": 1,
    "performance_rating": 1,
    "_id": 0 
}

print("\n--- High Performers ---")
results = employees_collection.find(query, projection)
for employee in results:
    print(f"Name: {employee['first_name']} {employee['last_name']}, "
          f"Department: {employee['department']}, "
          f"Salary: ${employee['salary']}, "
          f"Rating: {employee['performance_rating']}")

Sample Output 

Database connection successful. Proceeding with queries.

--- High Performers ---
Name: Kimberly Jenkins, Department: Operations, Salary: $84786, Rating: 4.7
Name: Adrienne Wilcox, Department: Finance, Salary: $134903, Rating: 4.1
Name: Jeffrey Hill, Department: IT, Salary: $113698, Rating: 4.6
Name: Richard Payne, Department: Human Resources, Salary: $90463, Rating: 4.3
Name: Tiffany Clark, Department: IT, Salary: $114034, Rating: 4.3
Name: Jose Rocha, Department: Finance, Salary: $118113, Rating: 4.5

Q2. **Experience-Based Filtering:** Find employees with 5-10 years of experience (inclusive) who earn between $70,000 and $120,000. Project only essential contact information (name, email, department).

In [ ]:

def find_experienced_employees():
    """
    Finds employees with 5-10 years of experience who earn between $70,000 and $120,000.
    Returns their name, email, and department.
    """
    if employees_collection is None:
        print("Database connection failed. Cannot perform query.")
        return


 #A compound query on a single field to specify a range. 
#It combines the $gte and $lte (less than or equal to) operators for years_experience.
    query = {
        "years_experience": {"$gte": 5, "$lte": 10}, 
        "salary": {"$gte": 70000, "$lte": 120000}
    }
    
    # The projection document is used to specify which fields should be returned.
    projection = {
        "first_name": 1, "last_name": 1, "email": 1, "department": 1, "_id": 0
    }
    

    print("\n--- Experienced Employees (5-10 years) ---")
    results = employees_collection.find(query, projection)

    found_results = False
    for employee in results:
        print(f"Name: {employee['first_name']} {employee['last_name']}, "
              f"Email: {employee['email']}, "
              f"Department: {employee['department']}")
        found_results = True

    if not found_results:   
        print("No experienced employees found in the specified range.")

find_experienced_employees()

Sample Output 

--- Experienced Employees (5-10 years) ---
Name: Kenneth Morton, Email: kenneth.morton@johnson.net, Department: IT
Name: Reginald Alvarez, Email: reginald.alvarez@spence.com, Department: Finance
Name: Christina Crawford, Email: christina.crawford@harris-rice.com, Department: Human Resources
Name: Michael Jacobs, Email: michael.jacobs@gordon.com, Department: Sales
Name: Catherine Hamilton, Email: catherine.hamilton@patrick-ortega.com, Department: Engineering
Name: Tiffany Clark, Email: tiffany.clark@young.com, Department: IT
Name: Jose Rocha, Email: jose.rocha@ramos.com, Department: Finance

Q3. **Salary Range Analysis:** Find employees whose salary is **NOT** in the range of $60,000-$100,000. Show their full name (concatenated), current salary, and years of experience.

In [ ]:
def find_employees_outside_salary_range():
    """
    Finds employees whose salary is NOT in the $60,000-$100,000 range.
    Returns their full name, salary, and years of experience.
    """
    if employees_collection is None:
        print("Database connection failed. Cannot perform query.")
        return


#The $or operator is used to combine multiple conditions. 
# In this case, we are looking for employees who meet one of two criteria: 
# their salary is less than $60,000 OR their salary is greater than $100,000.
    query = {
        "$or": [{"salary": {"$lt": 60000}}, {"salary": {"$gt": 100000}}]
    }
    
    projection = {
        "first_name": 1, "last_name": 1, "salary": 1, "years_experience": 1, "_id": 0
    }

    print("\n--- Employees Outside Salary Range ($60k-$100k) ---")
    results = employees_collection.find(query, projection)
    for employee in results:
        full_name = f"{employee['first_name']} {employee['last_name']}"
        print(f"Name: {full_name}, Salary: ${employee['salary']}, Experience: {employee['years_experience']} years")

find_employees_outside_salary_range()


Sample Output 

--- Employees Outside Salary Range ($60k-$100k) ---
Name: Kenneth Morton, Salary: $116357, Experience: 10 years
Name: Jeffrey Gordon, Salary: $148206, Experience: 6 years
Name: Misty Stephens, Salary: $121343, Experience: 1 years
Name: Alison Nelson, Salary: $132503, Experience: 8 years
Name: Jennifer James, Salary: $135188, Experience: 9 years
Name: Bryan Brown, Salary: $146276, Experience: 1 years
Name: Charles Anderson, Salary: $52755, Experience: 14 years

Q4. **Recent Hires:** Find employees hired in the last 2 years with performance rating > 3.5. Return custom fields showing "full_name", "tenure_months", and "annual_salary".

In [ ]:
def find_recent_hires():
    """
    Finds employees hired in the last 2 years with a performance rating > 3.5.
    Returns custom fields showing "full_name", "tenure_months", and "annual_salary".
    """
    if employees_collection is None:
        print("Database connection failed. Cannot perform query.")
        return
    
    two_years_ago = datetime.now() - timedelta(days=2 * 365)
    
    pipeline = [ #pipeline for aggregation
        {
            "$match": { #The $match aggregation is like a find() query. It filters the documents that enter the pipeline.
                "hire_date": {"$gte": two_years_ago},
                "performance_rating": {"$gt": 3.5}
            }
        },
        {
            #The $project aggregation stage is used to reshape the documents.
            "$project": {
                "_id": 0,
                "full_name": { "$concat": ["$first_name", " ", "$last_name"] },
                #The $concat operator is used to combine the first_name and last_name into a single full_name field.
                "tenure_months": {
                    "$divide": [ #$divide and $subtract operators are used to calculate the tenure in months.
                        {"$subtract": [datetime.now(), "$hire_date"]},
                        1000 * 60 * 60 * 24 * 30.44
                    ]
                },
                "annual_salary": "$salary" #This renames the existing salary field to annual_salary.
            }
        }
    ]
    
    print("\n--- Recent High-Performing Hires ---")
    results = employees_collection.aggregate(pipeline)

    found_results = False
     # Check if results are found
    for employee in results:
        tenure = round(employee['tenure_months'])
        print(f"Name: {employee['full_name']}, "
              f"Tenure: {tenure} months, "
              f"Annual Salary: ${employee['annual_salary']}")
        found_results = True


    if not found_results:
        print("No recent high-performing hires found.") 
find_recent_hires()

--- Recent High-Performing Hires ---
Name: Bryan Brown, Tenure: 19 months, Annual Salary: $146276
Name: Kimberly Jenkins, Tenure: 3 months, Annual Salary: $84786
Name: Daniel Garcia, Tenure: 3 months, Annual Salary: $67248
Name: Sean Dunn, Tenure: 2 months, Annual Salary: $140405
Name: Johnathan Coleman, Tenure: 20 months, Annual Salary: $51467
Name: April Cooper, Tenure: 15 months, Annual Salary: $58205
Name: Scott Morris, Tenure: 6 months, Annual Salary: $88334